In [ ]:
import os
import csv
from IPython.display import display
import sys
import time
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import gensim
import random
import psycopg2
from gensim.models import Word2Vec
import multiprocessing
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import seaborn as sns


In [ ]:
def fetch_db_rows():
    # Named cursor => server side cursor. Without this, _all_ data from the
    # query will be fetched into memory, which defeats the purpose of streaming
    query="SELECT codes FROM concept_embeddings.codes_by_site_aws_pasc_78"
    with \
            psycopg2.connect(user="USERNAME",
                                  password="PASSWORD",
                                  host="HOST",
                                  port="5432",
                                  database="DATABASE") as conn, \
            conn.cursor(name='get_rows') as cur:
        cur.itersize = 5
        cur.arraysize = 5
        cur.execute(query)

        while True:
            rows_temp = cur.fetchmany()
            rows=[x[0].split(" ") for x in rows_temp]     
#            rows=[list(set(x)) for x in rows_temp2]
            for row in rows:
#                if (len(row)>1):
                    yield random.sample(row, len(row))
            if not rows:
                break

class SentencesIterator():
    def __init__(self, generator_function):
        self.generator_function = generator_function
        self.generator = self.generator_function()

    def __iter__(self):
        # reset the generator
        self.generator = self.generator_function()
        return self

    def __next__(self):
        result = next(self.generator)
        if result is None:
            raise StopIteration
        else:
            return result

In [ ]:
sentences = SentencesIterator(fetch_db_rows)

In [ ]:
model = Word2Vec(vector_size=200, window=5, min_count=3, workers=12, sg=1, negative=5, hs=0)
#model.save("word2vec.model")

In [ ]:
model.build_vocab(sentences, progress_per=1000000)

In [ ]:
model.corpus_count

In [ ]:
sentences = SentencesIterator(fetch_db_rows)

In [ ]:
model.train(sentences,total_examples=model.corpus_count, epochs=15)

In [ ]:
model.save('./model_all_sites_aws_320_neg')